In [14]:
# 🧩 1. Import necessary libraries
import requests
import json
import time
import pandas as pd
import os
from datetime import datetime

In [15]:
# 🔐 2. Set API endpoints and keys
GRAPHQL_URL = "https://api.kpler.marinetraffic.com/v2/vessels/graphql"
GRAPHQL_API_KEY = "RTZCQWF2ZHJpWWVheXJZTmZIenFNRXFzZTQzdDNaUEQ6MU1vZ0dqWUQtZHFMMXNTMnNER21CZzNzSjcyUUdzQ0hfSHBFWlFRTk9pT2NwVWpzeF9TclZ0QzNvU0NwMlJpQg=="
AIS_API_KEY = "9fefb8446c56fb0a729888d02f87476e4aed1b1d"
HISTORICAL_API_KEY = "5299b468cfb79b872d52de2e7d76cb0d7aae5d38"
PORTCALLS_API_KEY = "466dfc66648fd8ee1d6e282c160100439653a463"

In [16]:
# 📦 3. Fetch vessels by REGISTER_OWNER
def fetch_vessels(after_cursor=None):
    query = f"""
    query Vessels {{
        vessels(
            first: 1000
            where: {{
                filters: [
                    {{
                        field: "management.beneficialOwner.current.name"
                        op: LIKE
                        values: ["AASEN SHIPPING%"]
                    }}
                ]
                operator: OR
            }}
            after: {json.dumps(after_cursor)}
        ) {{
            nodes {{
                identifier {{
                    imo
                }}
            }}
            pageInfo {{
                hasNextPage
                endCursor
            }}
        }}
    }}
    """

    headers = {
        "Authorization": f"Basic {GRAPHQL_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(GRAPHQL_URL, json={"query": query}, headers=headers)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None

    return response.json()

In [17]:
# 📥 4. Loop through pages and gather IMO list
imo_list = []
after_cursor = None

while True:
    data = fetch_vessels(after_cursor)
    if not data:
        break

    vessels = data["data"]["vessels"]["nodes"]
    for vessel in vessels:
        imo = vessel["identifier"].get("imo")
        if imo:
            imo_list.append(imo)

    page_info = data["data"]["vessels"]["pageInfo"]
    if page_info["hasNextPage"]:
        after_cursor = page_info["endCursor"]
    else:
        break

print(f"✅ Found {len(imo_list)} vessels. First 5 IMOs: {imo_list[:5]}")

✅ Found 7 vessels. First 5 IMOs: [9060778, 9101546, 9147136, 9321407, 9433389]


In [18]:
# 📡 5. Fetch live AIS positions from MT Export API
def fetch_ais_data(api_key, imo_list, timespan=1440, buffer_time=1):
    url_template = f'https://services.marinetraffic.com/api/exportvessel/{api_key}/v:6/timespan:{timespan}/imo:{{imo}}/protocol:jsono'
    all_ais_data = []

    for idx, imo in enumerate(imo_list, start=1):
        print(f"[{idx}/{len(imo_list)}] Fetching AIS for IMO: {imo}")
        try:
            response = requests.get(url_template.format(imo=imo))
            if response.ok:
                data = response.json()
                if isinstance(data, list):
                    for record in data:
                        record['IMO'] = imo
                        all_ais_data.append(record)
                else:
                    print(f"⚠️ Unexpected format for IMO {imo}")
            else:
                print(f"❌ Failed for IMO {imo}: {response.status_code}")
        except Exception as e:
            print(f"❌ Exception for IMO {imo}: {e}")

        if idx < len(imo_list):
            time.sleep(buffer_time)

    if all_ais_data:
        df = pd.DataFrame(all_ais_data)
        if {'IMO', 'SHIPNAME', 'LAT', 'LON', 'TIMESTAMP'}.issubset(df.columns):
            return df[['IMO', 'SHIPNAME', 'LAT', 'LON', 'TIMESTAMP']]
        else:
            return df
    else:
        print("⚠️ No AIS data fetched.")
        return pd.DataFrame()

In [19]:
# 🔍 6. Call the function and show results
df_ais = fetch_ais_data(AIS_API_KEY, imo_list)
print(f"✅ Retrieved {len(df_ais)} AIS position records.")
df_ais.head()

[1/7] Fetching AIS for IMO: 9060778
[2/7] Fetching AIS for IMO: 9101546
[3/7] Fetching AIS for IMO: 9147136
[4/7] Fetching AIS for IMO: 9321407
[5/7] Fetching AIS for IMO: 9433389
[6/7] Fetching AIS for IMO: 9904766
[7/7] Fetching AIS for IMO: 9904869
✅ Retrieved 7 AIS position records.


,IMO,SHIPNAME,LAT,LON,TIMESTAMP
0,9060778,AASLI,54.049202,-3.166848,2025-05-21T05:02:13
1,9101546,AASNES,55.719112,12.633556,2025-05-21T05:01:55
2,9147136,AASTIND,51.313572,3.831798,2025-05-21T05:00:10
3,9321407,AASTUN,54.095207,11.114382,2025-05-21T05:01:20
4,9433389,AASVIK,51.467525,0.257450,2025-05-21T04:59:44


In [20]:
# 💾 7. Save to CSV file
if not df_ais.empty:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    output_filename = f"ais_positions_pdvsa_{timestamp}.csv"
    df_ais.to_csv(output_filename, index=False)
    print(f"📁 Data saved to: {output_filename}")
else:
    print("⚠️ No data to save.")

📁 Data saved to: ais_positions_pdvsa_2025-05-21_13-04.csv


In [21]:
# 📡 8. Fetch and append historical AIS per IMO
def fetch_historical_ais_and_save(api_key, imo_list, output_path, days=180, period="daily", msgtype="simple", buffer_time=60):
    url_template = f'https://services.marinetraffic.com/api/exportvesseltrack/{api_key}/v:3/days:{days}/period:{period}/imo:{{imo}}/msgtype:{msgtype}/protocol:jsono'

    # 🧾 Create CSV with headers if it doesn't exist
    if not os.path.exists(output_path):
        with open(output_path, 'w', newline='') as f:
            pd.DataFrame(columns=['IMO', 'LAT', 'LON', 'TIMESTAMP']).to_csv(f, index=False)

    for idx, imo in enumerate(imo_list, start=1):
        print(f"[{idx}/{len(imo_list)}] Fetching and saving AIS for IMO: {imo}")
        try:
            response = requests.get(url_template.format(imo=imo))
            if response.ok:
                data = response.json()
                if isinstance(data, list) and data:
                    for record in data:
                        record['IMO'] = imo
                    df = pd.DataFrame(data)
                    
                    # ✍️ Save only if essential columns exist
                    if {'IMO', 'LAT', 'LON', 'TIMESTAMP'}.issubset(df.columns):
                        df[['IMO', 'LAT', 'LON', 'TIMESTAMP']].to_csv(output_path, mode='a', header=False, index=False)
                        print(f"✅ Saved {len(df)} records for IMO {imo}")
                    else:
                        print(f"⚠️ Skipped saving due to missing fields for IMO {imo}")
                else:
                    print(f"⚠️ No data returned for IMO {imo}")
            else:
                print(f"❌ Failed request for IMO {imo}: {response.status_code}")
        except Exception as e:
            print(f"❌ Exception for IMO {imo}: {e}")

        if idx < len(imo_list):
            time.sleep(buffer_time)

    # ✅ Return final DataFrame
    return pd.read_csv(output_path)

In [22]:
# 🔍 9. Call the historical AIS fetcher and preview
output_hist_file = f"historical_ais_pdvsa_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv"
df_hist_ais = fetch_historical_ais_and_save(
    api_key=HISTORICAL_API_KEY,
    imo_list=imo_list,
    output_path=output_hist_file,
    days=180,
    period="daily",
    buffer_time=60
)

print(f"✅ Retrieved {len(df_hist_ais)} historical AIS position records.")

# 💾 10. Already saved inside function, but you can log confirmation
if not df_hist_ais.empty:
    print(f"📁 Historical AIS data confirmed saved to: {output_hist_file}")
else:
    print("⚠️ No historical data to save.")

[1/7] Fetching and saving AIS for IMO: 9060778
✅ Saved 181 records for IMO 9060778
[2/7] Fetching and saving AIS for IMO: 9101546
✅ Saved 180 records for IMO 9101546
[3/7] Fetching and saving AIS for IMO: 9147136
✅ Saved 181 records for IMO 9147136
[4/7] Fetching and saving AIS for IMO: 9321407
✅ Saved 181 records for IMO 9321407
[5/7] Fetching and saving AIS for IMO: 9433389
✅ Saved 181 records for IMO 9433389
[6/7] Fetching and saving AIS for IMO: 9904766
✅ Saved 181 records for IMO 9904766
[7/7] Fetching and saving AIS for IMO: 9904869
✅ Saved 181 records for IMO 9904869
✅ Retrieved 1266 historical AIS position records.
📁 Historical AIS data confirmed saved to: historical_ais_pdvsa_2025-05-21_13-04.csv


In [23]:
# 📆 11. Derive dynamic date range from historical AIS DataFrame
def get_dynamic_date_range(df_hist_ais):
    if df_hist_ais.empty:
        raise ValueError("🚫 Historical AIS DataFrame is empty. Cannot derive dynamic date range.")

    df_hist_ais['TIMESTAMP'] = pd.to_datetime(df_hist_ais['TIMESTAMP'], errors='coerce')
    fromdate = df_hist_ais['TIMESTAMP'].min().strftime("%Y-%m-%d %H:%M")
    todate = df_hist_ais['TIMESTAMP'].max().strftime("%Y-%m-%d %H:%M")
    print(f"📆 Dynamic Date Range — From: {fromdate} | To: {todate}")
    return fromdate, todate

In [24]:
def fetch_port_calls_and_save(
    api_key,
    imo_list,
    output_path,
    df_hist_ais,
    msgtype="extended",
    buffer_time=120
):
    fromdate, todate = get_dynamic_date_range(df_hist_ais)

    # Add LOAD_STATUS to required columns
    required_columns = ['IMO', 'SHIPNAME', 'TIMESTAMP_UTC', 'MOVE_TYPE', 'PORT_ID', 'PORT_NAME', 'LOAD_STATUS']

    if not os.path.exists(output_path):
        pd.DataFrame(columns=required_columns).to_csv(output_path, index=False)
        print(f"📄 Created new CSV file with headers: {output_path}")

    url_template = (
        f'https://services.marinetraffic.com/api/portcalls/{api_key}/v:6/'
        f'fromdate:{fromdate}/todate:{todate}/imo:{{imo}}/msgtype:{msgtype}/protocol:jsono'
    )

    load_status_map = {
        0: "N/A",
        1: "In Ballast",
        2: "Partially Laden",
        3: "Fully Laden"
    }

    for idx, imo in enumerate(imo_list, start=1):
        print(f"[{idx}/{len(imo_list)}] Fetching Port Calls for IMO: {imo}")

        try:
            response = requests.get(url_template.format(imo=imo))
            if response.ok:
                data = response.json()
                if isinstance(data, list) and data:
                    for record in data:
                        record['IMO'] = imo

                        # Normalize raw_status safely:
                        raw_status = record.get('LOAD_STATUS', 0)

                        # Attempt to convert to int, fallback to 0
                        try:
                            raw_status_int = int(raw_status)
                        except (ValueError, TypeError):
                            raw_status_int = 0

                        # Map to human-readable string
                        record['LOAD_STATUS'] = load_status_map.get(raw_status_int, "N/A")

                    df = pd.DataFrame(data)

                    if set(required_columns).issubset(df.columns):
                        df[required_columns].to_csv(output_path, mode='a', header=False, index=False)
                        print(f"✅ Saved {len(df)} port call records for IMO {imo}")
                    else:
                        print(f"⚠️ Missing required fields — skipped saving for IMO {imo}")
                else:
                    print(f"⚠️ No port call data for IMO {imo}")
            else:
                print(f"❌ Failed request for IMO {imo}: {response.status_code}")
        except Exception as e:
            print(f"❌ Exception for IMO {imo}: {e}")

        if idx < len(imo_list):
            time.sleep(buffer_time)

In [25]:
# 📁 13. Output file path
output_csv = f"port_calls_pdvsa_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv"

# 📡 14. Run the fetcher with dynamic dates
fetch_port_calls_and_save(
    api_key=PORTCALLS_API_KEY,
    imo_list=imo_list,
    output_path=output_csv,
    df_hist_ais=df_hist_ais,  # Pass in the historical AIS dataframe
    buffer_time=60  # ⏳ Adjustable delay
)

📆 Dynamic Date Range — From: 2024-11-22 05:13 | To: 2025-05-21 00:54
📄 Created new CSV file with headers: port_calls_pdvsa_2025-05-21_13-10.csv
[1/7] Fetching Port Calls for IMO: 9060778
✅ Saved 196 port call records for IMO 9060778
[2/7] Fetching Port Calls for IMO: 9101546
✅ Saved 175 port call records for IMO 9101546
[3/7] Fetching Port Calls for IMO: 9147136
✅ Saved 174 port call records for IMO 9147136
[4/7] Fetching Port Calls for IMO: 9321407
✅ Saved 266 port call records for IMO 9321407
[5/7] Fetching Port Calls for IMO: 9433389
✅ Saved 304 port call records for IMO 9433389
[6/7] Fetching Port Calls for IMO: 9904766
✅ Saved 149 port call records for IMO 9904766
[7/7] Fetching Port Calls for IMO: 9904869
✅ Saved 199 port call records for IMO 9904869


In [27]:
# 15. Combine and Enrich All Datasets
from glob import glob
from datetime import datetime
import pandas as pd

def combine_and_enrich_datasets():
    latest_ais = sorted(glob("ais_positions_pdvsa_*.csv"))[-1]
    latest_hist = sorted(glob("historical_ais_pdvsa_*.csv"))[-1]
    latest_ports = sorted(glob("port_calls_pdvsa_*.csv"))[-1]

    print(f"🔄 Combining files:\n- {latest_ais}\n- {latest_hist}\n- {latest_ports}")

    df_live = pd.read_csv(latest_ais, parse_dates=["TIMESTAMP"])
    df_hist = pd.read_csv(latest_hist, parse_dates=["TIMESTAMP"])
    df_ports = pd.read_csv(latest_ports, parse_dates=["TIMESTAMP_UTC"])

    # Combine live and historical AIS data
    df_all_positions = pd.concat([df_live, df_hist], ignore_index=True)
    df_all_positions.sort_values(by=["IMO", "TIMESTAMP"], inplace=True)

    # 🔧 Fix missing SHIPNAME: Prefer from AIS, fallback to port_calls
    if "SHIPNAME" not in df_all_positions.columns:
        df_all_positions["SHIPNAME"] = None

    # Fill SHIPNAME using AIS data grouped by IMO
    df_all_positions["SHIPNAME"] = df_all_positions.groupby("IMO")["SHIPNAME"].transform(
        lambda x: x.ffill().bfill()
    )

    # Fallback: use SHIPNAME from port_calls if still missing
    if "SHIPNAME" in df_ports.columns:
        missing_names = df_all_positions["SHIPNAME"].isna()
        df_all_positions.loc[missing_names, "SHIPNAME"] = (
            df_all_positions[missing_names]
            .merge(df_ports[["IMO", "SHIPNAME"]].dropna().drop_duplicates(), on="IMO", how="left")["SHIPNAME_y"]
        )

    # 🧠 Add enrichment columns
    df_all_positions["LAST_PORT"] = None
    df_all_positions["LAST_PORT_ID"] = None
    df_all_positions["LAST_PORT_OP"] = None
    df_all_positions["NEXT_PORT"] = None
    df_all_positions["NEXT_PORT_ID"] = None
    df_all_positions["NEXT_PORT_OP"] = None

    for imo in df_all_positions["IMO"].unique():
        df_ais_vessel = df_all_positions[df_all_positions["IMO"] == imo]
        df_ports_vessel = df_ports[df_ports["IMO"] == imo]

        for idx, ais_row in df_ais_vessel.iterrows():
            position_time = ais_row["TIMESTAMP"]

            # Match Departure (LAST_PORT)
            departures = df_ports_vessel[
                (df_ports_vessel["MOVE_TYPE"] == 1) &
                (df_ports_vessel["TIMESTAMP_UTC"] <= position_time)
            ]
            if not departures.empty:
                last_dep = departures.sort_values(by="TIMESTAMP_UTC").iloc[-1]
                df_all_positions.at[idx, "LAST_PORT"] = last_dep["PORT_NAME"]
                df_all_positions.at[idx, "LAST_PORT_ID"] = last_dep["PORT_ID"]
                df_all_positions.at[idx, "LAST_PORT_OP"] = last_dep.get("LOAD_STATUS", "NA")

            # Match Arrival (NEXT_PORT)
            arrivals = df_ports_vessel[
                (df_ports_vessel["MOVE_TYPE"] == 0) &
                (df_ports_vessel["TIMESTAMP_UTC"] >= position_time)
            ]
            if not arrivals.empty:
                next_arr = arrivals.sort_values(by="TIMESTAMP_UTC").iloc[0]
                df_all_positions.at[idx, "NEXT_PORT"] = next_arr["PORT_NAME"]
                df_all_positions.at[idx, "NEXT_PORT_ID"] = next_arr["PORT_ID"]
                df_all_positions.at[idx, "NEXT_PORT_OP"] = next_arr.get("LOAD_STATUS", "NA")

    output_name = f"combined_ais_pdvsa_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv"
    df_all_positions.to_csv(output_name, index=False)
    print(f"✅ Enriched dataset saved to: {output_name}")

# 🧠 Run the combining process
combine_and_enrich_datasets()


🔄 Combining files:
- ais_positions_pdvsa_2025-05-21_13-04.csv
- historical_ais_pdvsa_2025-05-21_13-04.csv
- port_calls_pdvsa_2025-05-21_13-10.csv
✅ Enriched dataset saved to: combined_ais_pdvsa_2025-05-21_13-46.csv
